In [4]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import tensorboardX as tbx

%matplotlib inline

In [6]:
image_size = 286
crop_size = 256
batch_size = 1

lr = 0.0002
betas = [0.5, 0.999]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class UnPairedDataset(Dataset):
    def __init__(self, is_train):
        
    def __len__(self):
        
    def forward(self):
        

IndentationError: expected an indented block (<ipython-input-7-a56904387e45>, line 4)

In [ ]:
train_dataset = UnPaireddataset(is_train=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [10]:
class ResNetBlock(nn.Module):
    def __init__(self, channel):
        super(ResNetBlock, self).__init__()
        self.conv_block = nn.Sequential(
                            nn.Conv2d(channel, channel, kernel_size=3, stride=1, padding=1),
                            nn.InstanceNorm2d(channel),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(channel, channel, kernel_size=3, stride=1, padding=1),
                            nn.InstanceNorm2d(channel),
                          )
        
    def forward(self, x):
        out = x + self.conv_block(x)
        return out

In [19]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(True),
        
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(True),
            
            ResNetBlock(128),
            ResNetBlock(128),
            ResNetBlock(128),
            ResNetBlock(128),
            ResNetBlock(128),
            ResNetBlock(128),
            
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(True),
            
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        
        self.layers.apply(self._init_weights)
    
    def forward(self, x):
        return self.layers(x)
        
    def _init_weights(self, m):
        class_name = m.__class__.__name__
        if class_name.find('Conv') != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [20]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(64),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 1, kernel_size=4, stride=1, padding=1)
        )
        
        self.layers.apply(self._init_weights)
        
    def forward(self, x):
        return self.layers(x)
    
    def _init_weights(self, m):
        class_name = m.__class__.__name__
        if class_name.find('Conv') != -1:
            nn.init.normal(m.weight.data, 0.0, 0.02)